In [1]:
%load_ext autoreload
%autoreload 2

import torch
import matplotlib.pyplot as plt
import numpy as np
import yaml
import imageio.v3 as iio


In [2]:
config_file = './configs/lego.yaml'
with open(config_file, 'r') as file:
    config = yaml.safe_load(file)

In [3]:
from utils.Renderer import Renderer

renderer = Renderer(config['renderer'])

In [4]:
from utils.DataLoaderBlender import DataLoaderBlender

data_loader = DataLoaderBlender(config['data'])

poses, focal, H, W = data_loader.getDataset('test', exclude_imgs=True)

In [5]:
from models.Nerf import Nerf
from utils.Inference import Inference
import yaml
device =  'cuda' if torch.cuda.is_available() else 'cpu'
# device = 'cpu'
print(device)
model_sparse = Nerf(config['model']['L_xyz'],config['model']['L_dir'])
model_sparse.to(device)
model_fine = Nerf(config['model']['L_xyz'],config['model']['L_dir'])
model_fine.to(device)
models = (model_sparse, model_fine)
inference = Inference(models=models, 
                    checkpoint_path="nerf_lego_pt3d28072023234920.pt", 
                    device=device, renderer=renderer, dir_path=config['inference']['inference_folder'])


cuda
9
9
epoch
model_fine_state_dict
model_sparse_state_dict
optimizer_state_dict
train_loss_history
90


In [6]:
device = 'cpu'
if torch.cuda.is_available():
    device = torch.cuda.current_device()

In [7]:
from utils.Camera import Camera

cameras = []
# H, W = images[0].shape[:2]

for i in range(poses.shape[0]):
    cameras.append(Camera(H, W, poses[i], focal, device))

In [ ]:
for i in range(0, len(cameras), config['inference']['step']):
    file_name = f'r_{i:03d}.png'
    inference.eval_and_save(cameras[i], file_name)
    print(file_name)

torch.Size([800, 800, 3]) torch.Size([800, 800, 3])
r_000.png
torch.Size([800, 800, 3]) torch.Size([800, 800, 3])
r_002.png
torch.Size([800, 800, 3]) torch.Size([800, 800, 3])
r_004.png
torch.Size([800, 800, 3]) torch.Size([800, 800, 3])
r_006.png
torch.Size([800, 800, 3]) torch.Size([800, 800, 3])
r_008.png
torch.Size([800, 800, 3]) torch.Size([800, 800, 3])
r_010.png
torch.Size([800, 800, 3]) torch.Size([800, 800, 3])
r_012.png
torch.Size([800, 800, 3]) torch.Size([800, 800, 3])
r_014.png
torch.Size([800, 800, 3]) torch.Size([800, 800, 3])
r_016.png
torch.Size([800, 800, 3]) torch.Size([800, 800, 3])
r_018.png
torch.Size([800, 800, 3]) torch.Size([800, 800, 3])
r_020.png
torch.Size([800, 800, 3]) torch.Size([800, 800, 3])
r_022.png
torch.Size([800, 800, 3]) torch.Size([800, 800, 3])
r_024.png
torch.Size([800, 800, 3]) torch.Size([800, 800, 3])
r_026.png
torch.Size([800, 800, 3]) torch.Size([800, 800, 3])
r_028.png
torch.Size([800, 800, 3]) torch.Size([800, 800, 3])
r_030.png
torch.Si